# Material Optical Properties

This notebook is a deep dive into the optical constants available in the COMPASS material
database. We will explore:

1. Dielectric materials used in pixel stacks (SiO2, Si3N4, HfO2, TiO2).
2. Silicon: refractive index, absorption coefficient, and absorption depth.
3. Color filter transmittance for R, G, B channels.
4. Tungsten (metal grid): high extinction and skin depth.
5. Registering a custom material.
6. Comparing the Sellmeier and Cauchy dispersion models.

All wavelengths are in micrometers (um) following COMPASS conventions. Plots use nanometers
for readability.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from compass.materials.database import MaterialDB, MaterialData
from compass.core.units import um_to_nm, nm_to_um

In [ ]:
db = MaterialDB()
wavelengths = np.linspace(0.38, 0.78, 300)  # visible range, um
wl_nm = um_to_nm(wavelengths)  # for plotting

## 1. Dielectric Materials: SiO2, Si3N4, HfO2, TiO2

These transparent dielectrics are the building blocks of the pixel stack. They serve as
passivation layers (SiO2), anti-reflection coatings (Si3N4, HfO2), and high-index
spacers (TiO2). All have negligible absorption (k ~ 0) in the visible range, but differ
significantly in refractive index.

Higher refractive index materials like TiO2 (n ~ 2.3) provide stronger light bending
and better anti-reflection performance in thin-film stacks.

In [ ]:
dielectric_mats = ["sio2", "si3n4", "hfo2", "tio2"]
dielectric_labels = ["SiO$_2$", "Si$_3$N$_4$", "HfO$_2$", "TiO$_2$"]
dielectric_colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

fig, ax = plt.subplots(figsize=(8, 5))
for mat, label, color in zip(dielectric_mats, dielectric_labels, dielectric_colors):
    n_vals = np.array([db.get_nk(mat, wl)[0] for wl in wavelengths])
    ax.plot(wl_nm, n_vals, color=color, linewidth=2, label=label)

ax.set_xlabel("Wavelength (nm)", fontsize=12)
ax.set_ylabel("Refractive Index n", fontsize=12)
ax.set_title("Refractive Index of Dielectric Materials", fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(380, 780)
fig.tight_layout()
plt.show()

# Print values at 550 nm
print("Refractive index at 550 nm:")
for mat, label in zip(dielectric_mats, dielectric_labels):
    n, k = db.get_nk(mat, 0.55)
    print(f"  {label:>10}: n = {n:.4f}, k = {k:.6f}")

## 2. Silicon: n, k, Absorption Coefficient, and Absorption Depth

Silicon has a large refractive index (n ~ 3.5-5.5 in the visible) and wavelength-dependent
absorption. The absorption coefficient is:

$$\alpha = \frac{4\pi k}{\lambda}$$

and the characteristic absorption depth (1/e penetration) is:

$$d_{abs} = \frac{1}{\alpha} = \frac{\lambda}{4\pi k}$$

Blue light is absorbed in a thin layer near the surface, while red and NIR photons
penetrate deep into the silicon -- this is why back-side illuminated (BSI) sensors need
sufficient silicon thickness to efficiently capture long-wavelength photons.

In [ ]:
wl_si = np.linspace(0.38, 1.1, 400)  # extend into NIR
wl_si_nm = um_to_nm(wl_si)

n_si = np.array([db.get_nk("silicon", wl)[0] for wl in wl_si])
k_si = np.array([db.get_nk("silicon", wl)[1] for wl in wl_si])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))

ax1.plot(wl_si_nm, n_si, color="tab:blue", linewidth=2)
ax1.set_xlabel("Wavelength (nm)")
ax1.set_ylabel("n")
ax1.set_title("Silicon Refractive Index")
ax1.grid(True, alpha=0.3)

ax2.plot(wl_si_nm, k_si, color="tab:red", linewidth=2)
ax2.set_xlabel("Wavelength (nm)")
ax2.set_ylabel("k")
ax2.set_title("Silicon Extinction Coefficient")
ax2.set_yscale("log")
ax2.grid(True, alpha=0.3)

fig.tight_layout()
plt.show()

In [ ]:
# Absorption coefficient alpha = 4*pi*k / lambda (in 1/um)
alpha = 4.0 * np.pi * k_si / wl_si  # 1/um

# Absorption depth = 1/alpha (um)
# Avoid division by zero where k is very small
abs_depth = np.where(alpha > 1e-6, 1.0 / alpha, np.nan)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))

ax1.plot(wl_si_nm, alpha, color="tab:green", linewidth=2)
ax1.set_xlabel("Wavelength (nm)")
ax1.set_ylabel(r"$\alpha$ (1/um)")
ax1.set_title(r"Silicon Absorption Coefficient $\alpha = 4\pi k / \lambda$")
ax1.set_yscale("log")
ax1.grid(True, alpha=0.3)

ax2.plot(wl_si_nm, abs_depth, color="tab:purple", linewidth=2)
ax2.set_xlabel("Wavelength (nm)")
ax2.set_ylabel("Absorption Depth (um)")
ax2.set_title(r"Silicon Absorption Depth $1/\alpha$")
ax2.set_yscale("log")
ax2.set_ylim(0.01, 1000)
ax2.grid(True, alpha=0.3)

# Add reference lines for typical silicon thicknesses
for thickness, label in [(3.0, "3 um (BSI)"), (6.0, "6 um (thick BSI)")]:
    ax2.axhline(thickness, color="grey", linestyle="--", alpha=0.5)
    ax2.text(400, thickness * 1.15, label, fontsize=9, color="grey")

fig.tight_layout()
plt.show()

# Print absorption depths at key wavelengths
print("Absorption depth at key wavelengths:")
for wl_target in [0.45, 0.55, 0.65, 0.80, 1.00]:
    idx = np.argmin(np.abs(wl_si - wl_target))
    depth = abs_depth[idx]
    if np.isnan(depth):
        print(f"  {um_to_nm(wl_target):.0f} nm: >> 100 um (negligible absorption)")
    else:
        print(f"  {um_to_nm(wl_target):.0f} nm: {depth:.2f} um")

## 3. Color Filter Transmittance

Color filters are absorbing polymer layers placed above each pixel. Their transmittance
depends on the extinction coefficient `k` and the layer thickness `d`:

$$T(\lambda) = \exp\!\left(-\frac{4\pi k(\lambda)\, d}{\lambda}\right)$$

A typical color filter thickness is 0.6 um. The filter design aims for high transmittance
in the target band and strong absorption outside it.

In [ ]:
cf_thickness = 0.6  # um (typical color filter thickness)
cf_names = ["cf_red", "cf_green", "cf_blue"]
cf_labels = ["Red", "Green", "Blue"]
cf_colors = ["red", "green", "blue"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))

for name, label, color in zip(cf_names, cf_labels, cf_colors):
    k_cf = np.array([db.get_nk(name, wl)[1] for wl in wavelengths])
    # Transmittance: exp(-4*pi*k*d / lambda)
    transmittance = np.exp(-4.0 * np.pi * k_cf * cf_thickness / wavelengths)

    ax1.plot(wl_nm, k_cf, color=color, linewidth=2, label=label)
    ax2.plot(wl_nm, transmittance, color=color, linewidth=2, label=label)

ax1.set_xlabel("Wavelength (nm)")
ax1.set_ylabel("k")
ax1.set_title("Color Filter Extinction Coefficient")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel("Wavelength (nm)")
ax2.set_ylabel("Transmittance")
ax2.set_title(f"Color Filter Transmittance (d = {cf_thickness} um)")
ax2.set_ylim(0, 1.05)
ax2.legend()
ax2.grid(True, alpha=0.3)

fig.tight_layout()
plt.show()

## 4. Tungsten: Metal Grid Material

Tungsten is used as the material for metal light-guide grids in advanced pixel designs.
As a metal, it has very high extinction coefficient `k`, which means light is absorbed
within a very thin skin depth:

$$\delta = \frac{\lambda}{4\pi k}$$

This makes tungsten an effective optical barrier between neighbouring pixels.

In [ ]:
n_w = np.array([db.get_nk("tungsten", wl)[0] for wl in wavelengths])
k_w = np.array([db.get_nk("tungsten", wl)[1] for wl in wavelengths])

# Skin depth
skin_depth_nm = um_to_nm(wavelengths / (4.0 * np.pi * k_w))  # nm

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(wl_nm, n_w, color="tab:blue", linewidth=2)
axes[0].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("n")
axes[0].set_title("Tungsten -- n")
axes[0].grid(True, alpha=0.3)

axes[1].plot(wl_nm, k_w, color="tab:red", linewidth=2)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("k")
axes[1].set_title("Tungsten -- k")
axes[1].grid(True, alpha=0.3)

axes[2].plot(wl_nm, skin_depth_nm, color="tab:orange", linewidth=2)
axes[2].set_xlabel("Wavelength (nm)")
axes[2].set_ylabel("Skin Depth (nm)")
axes[2].set_title("Tungsten -- Skin Depth")
axes[2].grid(True, alpha=0.3)

fig.tight_layout()
plt.show()

print(f"Skin depth at 550 nm: {skin_depth_nm[np.argmin(np.abs(wl_nm - 550))]:.1f} nm")

## 5. Register a Custom Material

COMPASS supports several ways to add your own materials:

- **Constant**: fixed n, k -- suitable for simple models.
- **Cauchy**: n(lam) = A + B/lam^2 + C/lam^4 -- good for transparent dielectrics.
- **Sellmeier**: more accurate dispersion model using resonance terms.
- **CSV file**: fully tabulated n(lam), k(lam) data.

Let us register a hypothetical new anti-reflection coating material using the Cauchy model
and plot it alongside existing materials.

In [ ]:
# Register a custom Cauchy material: a hypothetical high-index AR coating
db.register_cauchy("custom_arc", A=2.05, B=0.015, C=0.001)

# Verify it is registered
print(f"Material 'custom_arc' registered: {db.has_material('custom_arc')}")

# Plot against existing dielectrics
fig, ax = plt.subplots(figsize=(8, 5))

for mat, label, color in zip(
    ["sio2", "si3n4", "hfo2", "custom_arc"],
    ["SiO$_2$", "Si$_3$N$_4$", "HfO$_2$", "Custom ARC"],
    ["tab:blue", "tab:orange", "tab:green", "tab:red"],
):
    n_vals = np.array([db.get_nk(mat, wl)[0] for wl in wavelengths])
    ax.plot(wl_nm, n_vals, color=color, linewidth=2, label=label,
            linestyle="--" if mat == "custom_arc" else "-")

ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("Refractive Index n")
ax.set_title("Custom Material vs Built-in Dielectrics")
ax.legend()
ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()

# Print updated material list
print(f"\nAll materials: {db.list_materials()}")

## 6. Sellmeier vs Cauchy Model Comparison

Both Cauchy and Sellmeier are dispersion models for transparent materials. The **Cauchy**
model is a simple polynomial in 1/lam^2:

$$n(\lambda) = A + \frac{B}{\lambda^2} + \frac{C}{\lambda^4}$$

The **Sellmeier** equation is more physically motivated, based on resonance:

$$n^2(\lambda) = 1 + \sum_i \frac{B_i \lambda^2}{\lambda^2 - C_i}$$

For SiO2, COMPASS uses the Sellmeier model (three resonance terms). Let us fit a
Cauchy model to the same data and compare.

In [ ]:
# SiO2 Sellmeier (already in the database)
n_sellmeier = np.array([db.get_nk("sio2", wl)[0] for wl in wavelengths])

# Fit a Cauchy model to SiO2 by choosing A, B, C to approximate the Sellmeier curve.
# We do a simple least-squares fit: n = A + B/lam^2 + C/lam^4
lam2 = wavelengths ** 2
lam4 = wavelengths ** 4
# Design matrix: [1, 1/lam^2, 1/lam^4]
X = np.column_stack([np.ones_like(wavelengths), 1.0 / lam2, 1.0 / lam4])
# Solve for [A, B, C]
coeffs, _, _, _ = np.linalg.lstsq(X, n_sellmeier, rcond=None)
A_fit, B_fit, C_fit = coeffs
print(f"Fitted Cauchy coefficients for SiO2:")
print(f"  A = {A_fit:.6f}")
print(f"  B = {B_fit:.6f}")
print(f"  C = {C_fit:.6f}")

# Register the fitted Cauchy model
db.register_cauchy("sio2_cauchy_fit", A=A_fit, B=B_fit, C=C_fit)
n_cauchy = np.array([db.get_nk("sio2_cauchy_fit", wl)[0] for wl in wavelengths])

# Plot both
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))

ax1.plot(wl_nm, n_sellmeier, color="tab:blue", linewidth=2, label="Sellmeier")
ax1.plot(wl_nm, n_cauchy, color="tab:red", linewidth=2, linestyle="--", label="Cauchy (fitted)")
ax1.set_xlabel("Wavelength (nm)")
ax1.set_ylabel("n")
ax1.set_title("SiO$_2$: Sellmeier vs Cauchy")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Residual
residual = (n_cauchy - n_sellmeier) * 1e4  # in units of 1e-4
ax2.plot(wl_nm, residual, color="tab:green", linewidth=2)
ax2.set_xlabel("Wavelength (nm)")
ax2.set_ylabel(r"$\Delta n \;(\times 10^{-4})$")
ax2.set_title("Cauchy - Sellmeier Residual")
ax2.axhline(0, color="grey", linestyle="--", alpha=0.5)
ax2.grid(True, alpha=0.3)

fig.tight_layout()
plt.show()

print(f"\nMax |residual|: {np.max(np.abs(residual)):.2f} x 1e-4")
print("The Cauchy model provides an excellent fit to SiO2 over the visible range.")

## Summary

Key takeaways from this material exploration:

| Material | n at 550 nm | k at 550 nm | Role in pixel |
|----------|:-----------:|:-----------:|---------------|
| SiO2     | ~1.46       | ~0          | Passivation, planarization, DTI fill |
| Si3N4    | ~2.02       | ~0          | Anti-reflection, high-index layer |
| HfO2     | ~1.97       | ~0          | BARL anti-reflection |
| TiO2     | ~2.43       | ~0          | High-index AR, light guide |
| Silicon  | ~4.08       | ~0.03       | Photodiode absorber |
| Tungsten | ~3.65       | ~3.08       | Metal grid light barrier |
| Color filters | ~1.55  | 0-0.18      | Wavelength-selective absorption |

Important physics:

- **Absorption depth** in silicon ranges from ~0.1 um (blue) to >10 um (NIR), driving the
  requirement for BSI sensor thickness.
- **Color filter transmittance** depends exponentially on k and thickness -- thicker filters
  give better color separation but reduce signal.
- **Tungsten skin depth** of ~14 nm means even thin metal grids are effective light barriers.
- The **Cauchy model** is a good approximation to **Sellmeier** for low-dispersion dielectrics
  in the visible range (residuals < 1e-4 for SiO2).

### Next steps

- **03_signal_chain.ipynb** -- Use these materials together with illuminants, scene
  reflectances, and module optics to simulate the full signal chain.